# Large Language Models for Education

In this notebook, we demonstrate how to use large language models (LLMs) for use cases in education.  LLMs can be used for tasks such as summarization, question-answering, or the generation of question & answer pairs.

The first part of this notebook sets up a SageMaker endpoint, using a **FLAN T5** model, on a single `p3.2xlarge` instance.  Next we show how the endpoint can be queried, and some subroutines are added to demonstrate prompts for summarization, question-answering and the generation of question & answer pairs.

The final section is split into four demos on querying a university-level text on quantum computing, an example syllabus, a guide on "design thinking", and an ebook text from [Project Gutenberg](https://www.gutenberg.org/).

https://github.com/ngnatk/amazon-sagemaker-immersion-day-for-research/tree/main/10.%20Generative_AI/2.%20Education_QnA

## 1.Setting up the SageMaker Endpoint using SageMaker Jumpstart

### 1.1 Install Python Dependencies

In [3]:
%%capture
!pip install --upgrade pip

In [4]:
%%capture
!pip install -U langchain --quiet
!pip install -U PyPDF2 --quiet
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade sagemaker --quiet

### 1.2 Deploying a SageMaker Endpoint

In [5]:
import sagemaker, boto3, json
from sagemaker import image_uris, instance_types, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.session import Session
from sagemaker.utils import name_from_base
from IPython.display import display, HTML

In [6]:
# Create the folder where the model weights will be stored
!mkdir -p download_dir

In [7]:
def get_sagemaker_session(local_download_dir) -> sagemaker.Session:
    """Return the SageMaker session."""

    sagemaker_client = boto3.client(
        service_name="sagemaker", region_name=boto3.Session().region_name
    )

    session_settings = sagemaker.session_settings.SessionSettings(
        local_download_dir=local_download_dir
    )

    # the unit test will ensure you do not commit this change
    session = sagemaker.session.Session(
        sagemaker_client=sagemaker_client, settings=session_settings
    )

    return session

In [8]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

model_id, model_version = "huggingface-text2text-flan-t5-xl", "*"
_model_env_variable_map = {
    "huggingface-text2text-flan-t5-xl": {"MMS_DEFAULT_WORKERS_PER_MODEL": "1"},
}

endpoint_name = name_from_base(f"jumpstart-example-{model_id}")
instance_type = 'ml.p3.2xlarge'

In [9]:
# Retrieve the inference docker container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

# Create the SageMaker model instance
if model_id in _model_env_variable_map:
    # For those large models, we already repack the inference script and model
    # artifacts for you, so the `source_dir` argument to Model is not required.
    model = Model(
        image_uri=deploy_image_uri,
        model_data=model_uri,
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        env=_model_env_variable_map[model_id],
    )
else:
    model = Model(
        image_uri=deploy_image_uri,
        source_dir=deploy_source_uri,
        model_data=model_uri,
        entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        sagemaker_session=get_sagemaker_session("download_dir"),
    )

In [10]:
%%time
# deploy the Model. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
print(f'Deploying endpoint {endpoint_name} on 1 x {instance_type} (this will take approximately 6-8 minutes)')
model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    predictor_cls=Predictor,
    endpoint_name=endpoint_name,
)

Deploying endpoint jumpstart-example-huggingface-text2text-2023-06-05-03-18-35-650 on 1 x ml.p3.2xlarge (this will take approximately 6-8 minutes)
-----------!CPU times: user 290 ms, sys: 13.1 ms, total: 303 ms
Wall time: 6min 3s


In [11]:
print(f'Successfully deployed endpoint {endpoint_name} on 1 x {instance_type}')

Successfully deployed endpoint jumpstart-example-huggingface-text2text-2023-06-05-03-18-35-650 on 1 x ml.p3.2xlarge


## 2. Subroutines for text extraction and for querying the endpoint

In [12]:
import pandas as pd
import PyPDF2
import re
import requests

from bs4 import BeautifulSoup

pd.set_option('max_colwidth', 80)  # Set max column width for displaying Pandas Dataframes
QNA_OUTPUT_STYLE = 'HTML'

### 2.1 Extract Pages from PDF file

In [13]:
def extract_pages(pdf_file, max_pages=25):
    pages = []
    with open(pdf_file, 'rb') as f:
        for i, page in enumerate(PyPDF2.PdfReader(f).pages):
            if i == max_pages:
                break
            pages.append(page.extract_text())
    return pages

### 2.2 Download a text-based ebook from a URL

In [14]:
def download_book(url):
    r = requests.get(url)
    if r.status_code == 200:
        return r.content.decode('utf-8')
    else:
        print(f'Failed to download {url}. Status code = {r.status_code}')
        return None

### 2.3 Extract paragraphs from a web page

In [15]:
def get_website_text(url):
    r = requests.get(url)
    if r.status_code == 200:
        return r.text
    else:
        print(f'Failed to access {url}. Status code = {r.status_code}')
        return None
    
def extract_paragraphs_from_html(text):
    html = BeautifulSoup(text, 'html.parser')
    return [ p.text for p in html.body.select('p') ]

### 2.4 Generate texts, or query an endpoint with prompts

In [16]:
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
lightred, lightgreen, lightyellow, lightblue = '\033[91m', '\033[92m', '\033[93m', '\033[94m'
lightmagenta, lightcyan, reset = '\033[95m', '\033[96m', '\33[39m'

def query_endpoint_with_json_payload(encoded_json):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions['generated_texts']
    return generated_text

def generate_text_from_prompt(prompt, max_length=300, max_time=50, temperature=0.5,
                              top_k=None, top_p=None, do_sample=True, seed=None):
    payload = {
        "text_inputs": prompt,
        "max_length": max_length,
        "max_time": max_time,
        "temperature": temperature,
        "do_sample": do_sample
    }
    if top_k is not None:
        payload['top_k'] = top_k
    if top_p is not None:
        payload['top_p'] = top_p
    if seed is not None:
        payload['seed'] = seed

    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    return parse_response_multiple_texts(query_response)[0]

To further customize the outputs of the large language model, the following parameters are available:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_time:** The maximum amount of time you allow the computation to run for in seconds. Generation will still finish the current pass after allocated time has been passed. This setting can help to generate a response prior to endpoint invocation response time out errors.
* **num_return_sequences:** Number of output sequences returned. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **seed:** Fix the randomized state for reproducibility. If specified, it must be an integer.


In [17]:
def summarize(text, seed=None):
    return generate_text_from_prompt(
        f"""Summarize the following text in 100 words:\n\n{text}\n\nSummary:""",
        temperature=0.2,  # Low temperature for summarization
        seed=seed
    )

def ask(text, question, seed=None):
    return generate_text_from_prompt(
        f"""My text is:\n{text}\n\n{question}""",
        temperature=0.01,  # Lowest temperature for accuracy
        seed=seed
    )

def extract_question(text, seed=None):
    return generate_text_from_prompt(
        f"""EXTRACT QUESTIONS\nContext:\n{text}\nQuestion:""",
        temperature=1.0,  # Maximum temperature for creativity
        seed=seed
    )

def create_qna_pairs(text, n, output_style='HTML', qn_seed=None, ans_seed=None):
    questions = []
    answers = []

    for i in range(n):
        questions.append(extract_question(text, qn_seed))
        answers.append(ask(text, questions[i], ans_seed))
        if output_style == 'HTML':
            output = \
            f"""<b>{i+1}</b>. <b><font color=#FF7F50>Question</font></b>: {questions[i]}
            <b><font color=#FA8072>Answer</font></b>: {answers[i]}"""
            display(HTML(output))
        elif output_style == 'text':
            print(f"""{i+1}. {lightblue}{bold}Question{unbold}{reset}: {questions[i]} {lightcyan}{bold}Answer{unbold}{reset}: {answers[i]}""")
    if output_style == 'table':
        return pd.DataFrame({
            'Question': questions,
            'Answer': answers
        })

## 3. LLM Demos for Education

In this notebook, we use the following texts to demonstrate summarization tasks and the generation of question & answer pairs.

1. Quantum Computing from Wikipedia: https://en.wikipedia.org/wiki/Quantum_computing
<!-- 1. Quantum Computing and Quantum Information (by Nielsen & Chuang): https://michaelnielsen.org/qcqi/QINFO-book-nielsen-and-chuang-toc-and-chapter1-nov00.pdf (this is a sample chapter from [this website](https://michaelnielsen.org/qcqi/)) -->
2. Primary School Science Syllabus (by Ministry of Education, Singapore): https://www.moe.gov.sg/-/media/files/primary/syllabus/2023-primary-science.ashx
3. Design Thinking (by Michael Shanks, Stanford): https://web.stanford.edu/~mshanks/MichaelShanks/files/509554.pdf 
4. Winnie the Pooh (by Alan Alexander Milne): https://www.gutenberg.org/ebooks/67098.txt.utf-8
5. Attention is all you need (by Vaswani et al): https://arxiv.org/pdf/1706.03762.pdf

Note that for this notebook, we are using the Flan T5 XL model for simplicity and ease of deployment--additional fine tuning or using improved models would be required to get better results.

In [18]:
# Download pdfs and texts with the `curl` command. Flags used here are `-L` (allow redirects),
# `-s` (for silent mode) and `-o` (to specify the output file name).

# Quantum Computation and Quantum Information (by Nielsen & Chuang) (sample chapter)
# !curl -Ls https://michaelnielsen.org/qcqi/QINFO-book-nielsen-and-chuang-toc-and-chapter1-nov00.pdf -o nielsen-n-chuang-ch01.pdf

# Primary School Science Syllabus (by Ministry of Education, Singapore)
!curl -Ls https://www.moe.gov.sg/-/media/files/primary/syllabus/2023-primary-science.ashx -o 2023-primary-science.pdf

# Design Thinking (by Michael Shanks, Stanford)
!curl -Ls https://web.stanford.edu/~mshanks/MichaelShanks/files/509554.pdf -o design-thinking.pdf

# Winnie the Pooh (by Alan Alexander Milne)
!curl -LOs https://www.gutenberg.org/ebooks/67098.txt.utf-8

# Attention is all you need (by Vaswani et al)
!curl -Ls https://arxiv.org/pdf/1706.03762.pdf -o attention.pdf

### 3.1 Wikipedia Page on Quantum Computing

In [19]:
NCHARS = 400     # We will show just the first and last 400 characters of each extracted text. Increase this number for more context.
NQUESTIONS = 12  # The number of Q&A pairs that we will generate.

In [20]:
txt1_paragraphs = extract_paragraphs_from_html(
    get_website_text('https://en.wikipedia.org/wiki/Quantum_computing')
)
txt1 = '\n\n'.join(txt1_paragraphs[1:8])
print(f'{txt1[:NCHARS]}...\n\n...{txt1[-NCHARS:]}')

A quantum computer is a computer that exploits quantum mechanical phenomena.
At small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior using specialized hardware.
Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster than any ...

...s,[19] which drew significant attention to the field of quantum computing.[20]
In 1996, Grover's algorithm established a quantum speedup for the widely applicable unstructured search problem.[21][22] The same year, Seth Lloyd proved that quantum computers could simulate quantum systems without the exponential overhead present in classical simulations,[23] validating Feynman's 1982 conjecture.[24]



#### Key word Generation

In [21]:
KEY_WORDS = generate_text_from_prompt(
    f'FIND KEY WORDS\n\nContext:\n{txt1}\nKey Words:',
    seed=123
)
key_word_list = KEY_WORDS.split(', ')
print(KEY_WORDS)

quantum computing, qubit, superposition, quantum theory


In [22]:
# Same as above, but using Amazon Comprehend. Requires `ComprehendReaadOnly` permissions.

# txt_list = txt1_paragraphs[2:20]  # Max is 25 paragraphs
# try:
#     client = boto3.client('comprehend')
#     key_phrases = client.batch_detect_key_phrases(
#         TextList=txt_list,
#         LanguageCode='en'
#     )
# except client.exceptions.BatchSizeLimitExceededException as e:
#     print(f'{e}')
#     print(f'Your batch size is: {len(txt_list)}')
# except client.exceptions.TextSizeLimitExceededException as e:
#     print(f'{e}')
#     print(f'Your maximum input text size is: {max([ len(x) for x in txt_list ])}')
# except Exception as e:
#     print(f'Error {e}')
#     print('If this is an AccessDeniedException, you need to add `ComprehendReaadOnly` permissions to {aws_role}')
# else:    
#     phrases = set()
#     for x in key_phrases['ResultList']:
#         for p in x['KeyPhrases']:
#             if p['Score'] > 0.999:
#                 phrases.add(p['Text'])
#     print(','.join(phrases))

#### Summary of key points

For each of each of paragraphs 3-11, let's create a short summary

In [23]:
summary = []
for i, x in enumerate(txt1_paragraphs[2:12]):
    summary.append(f'{i+1}. {summarize(x[:1500])}')

In [24]:
HTML(
    '<h4>Key Points</h4>' + 
    '\n'.join([ f'<li>{x}</li>' for x in summary ])
)

In [25]:
# Let's try to summarize the 10 points above
summarize('\n'.join(summary))

'Quantum computing is the development of a computer that uses quantum physics to perform computations that are impossible for a classical computer.'

#### Checking for correct answers

In [26]:
prompt=f"""Context:{txt1}
What is quantum computing?"""
answer = generate_text_from_prompt(prompt, temperature=0.01)
print(answer)

A quantum computer is a computer that exploits quantum mechanical phenomena


In [27]:
prompt=f"""Context:{txt1}
Question: What is quantum computing?
Answer: {answer}
Student: Quantum computing is using computers with quantum dots
Is this answer correct?"""
print(generate_text_from_prompt(prompt, temperature=0.01))

no


In [28]:
prompt=f"""Context:{txt1}
Question: What is quantum computing?
Answer: {answer}
Student: Quantum computing involves using computers that make use of quantum mechanics
Is this answer correct?"""
print(generate_text_from_prompt(prompt, temperature=0.01))

yes


#### Generation of Question & Answer Pairs

In [29]:
create_qna_pairs(txt1, NQUESTIONS, output_style=QNA_OUTPUT_STYLE)

### 3.2 Primary School Science Syllabus (by Ministry of Education, Singapore)

In [30]:
primary_science = extract_pages('2023-primary-science.pdf')

In [31]:
txt2 = primary_science[15]  # Extract page 16
print(f'{txt2[:NCHARS]}...\n\n\n...{txt2[-NCHARS:]}')

 
15 
OFFICIAL (OPEN)  
 
 Interactions  
 
Interactions are the actions between and within living and non -living systems in the 
environment. Understanding these interactions helps us see relationships between the 
factors/variables in the environment. We can also appreciate the consequ ences of our 
actions and play our part in conservation. The essential takeaways and key inquiry 
questions fo...


...s, living 
and non -living things in the 
environment.   
• Interactions within the environment can 
have positive or negative impacts.  
• Conservation is important to ensure 
continuity of life and availability of 
resources.  
 • What are the types of 
interactions around us?  
• How do interactions affect the 
environment and us?  
• Why is it important for us to 
conserve the environment?  
 


In [32]:
create_qna_pairs(txt2, NQUESTIONS, output_style=QNA_OUTPUT_STYLE)

### 3.3 Design Thinking (by Michael Shanks, Stanford)

In [33]:
design_thinking = extract_pages('design-thinking.pdf')

In [34]:
txt3 = '\n\n'.join(design_thinking[1:4])
print(f'{txt3[:NCHARS]}...\n\n\n...{txt3[-NCHARS:]}')

WHAT is the Empathize mode
Empathy is the centerpiece of a human-centered design process.  The Empathize mode is 
the work you do to understand people, within the context of your design challenge.  It is your 
eﬀort to understand the way they do things and why, their physical and emotional needs, how 
they think about world, and what is meaningful to them. 
WHY empathize
As a design thinker, the p...


...t ideas that your team 
generated during brainstorming.  Carry the two or three ideas that receive the most 
votes forward into prototyping.  In this way, you preserve innovation potential by 
carrying multiple ideas forward—a radically diﬀerent approach than settling on the 
single idea that at least the majority of the team can agree upon.
Maximize your innovation potentialIdeate
PrototypeIdeate


In [35]:
create_qna_pairs(txt3, NQUESTIONS, output_style=QNA_OUTPUT_STYLE)

### 3.4 Winnie the Pooh (by Alan Alexander Milne)

In [36]:
winnie_the_pooh = download_book('https://www.gutenberg.org/ebooks/67098.txt.utf-8')

In [37]:
x = winnie_the_pooh.find('CHAPTER III')
txt4 = winnie_the_pooh[x:x+4000]  # Extract the first 4000 characters of chapter 3
print(txt4)

CHAPTER III

                   IN WHICH POOH AND PIGLET GO HUNTING
                        AND NEARLY CATCH A WOOZLE


The Piglet lived in a very grand house in the middle of a beech-tree,
and the beech-tree was in the middle of the forest, and the Piglet lived
in the middle of the house. Next to his house was a piece of broken
board which had: "TRESPASSERS W" on it. When Christopher Robin asked the
Piglet what it meant, he said it was his grandfather's name, and had
been in the family for a long time, Christopher Robin said you
_couldn't_ be called Trespassers W, and Piglet said yes, you could,
because his grandfather was, and it was short for Trespassers Will,
which was short for Trespassers William. And his grandfather had had two
names in case he lost one--Trespassers after an uncle, and William after
Trespassers.

"I've got two names," said Christopher Robin carelessly.

"Well, there you are, that proves it," said Piglet.

One fine winter's day when Piglet was brushing away the s

In [38]:
ask(txt4, "What is the storyline here?")

'The Piglet lived in a very grand house in the middle of a beech-tree, and the beech-tree was in the middle of the forest, and the Piglet lived in the middle of the house. Next to his house was a piece of broken board which had: "TRESPASSERS W" on it. When Christopher Robin asked the Piglet what it meant, he said it was his grandfather\'s name, and had been in the family for a long time. Christopher Robin said you _could_ be called Trespassers W, and Piglet said yes, you could, because his grandfather was, and it was short for Trespassers Will, which was short for Trespassers William. And his grandfather had had two names in case he lost one--Trespassers after an uncle, and William after Trespassers.'

In [39]:
ask(txt4, "Who is the main character?")

'Winnie-the-Pooh'

In [40]:
ask(txt4, "What happens at the end?")

'Pooh and Piglet nearly catch a Woozle.'

In [41]:
create_qna_pairs(txt4, NQUESTIONS, output_style=QNA_OUTPUT_STYLE)

### 3.5 Attention is all you need (by Vaswani et al)

In [42]:
attention = extract_pages('attention.pdf')

In [43]:
txt5 = '\n\n'.join(attention[1:3] + attention[9:10])  # We will use pages 1, 2, and 9
print(f'{txt5[:NCHARS]}...\n\n\n...{txt5[-NCHARS:]}')

transduction problems such as language modeling and machine translation [ 35,2,5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a se...


...g. arXiv preprint arXiv:1601.06733 , 2016.
[5]Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk,
and Yoshua Bengio. Learning phrase representations using rnn encoder-decoder for statistical
machine translation. CoRR , abs/1406.1078, 2014.
[6]Francois Chollet. Xception: Deep learning with depthwise separable convolutions. arXiv
preprint arXiv:1610.02357 , 2016.
10


#### Question Answering

In [44]:
ask(txt5, "What is the main gist of the paper?")

'We propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.'

In [45]:
ask(txt5, "What is the problem being solved?")

'The Transformer is the first transduction model relying entirely on attention to compute representations of its input and output without using sequence- aligned RNNs or convolution.'

In [46]:
ask(txt5, "What is the conclusion of the paper?")

'In this work, we presented the Transformer, the first sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention.'

In [47]:
chunk_size = len(txt5)//8
print(f'The text will be split up into chunks of {chunk_size} characters and summarized')

The text will be split up into chunks of 1202 characters and summarized


In [48]:
display(HTML('<h4>Key Points</h4>'))
summary = []
for i in range(8):
    x0 = i*chunk_size
    x1 = (i+1)*chunk_size
    line_summary = f'{i+1}. {summarize(txt5[x0:x1])}'
    display(HTML(line_summary))
    summary.append(line_summary)

In [49]:
create_qna_pairs(txt5, NQUESTIONS, output_style=QNA_OUTPUT_STYLE)

## Cleanup

Delete the SageMaker model and endpointDelete the SageMaker model and endpoint

In [50]:
model_predictor.delete_model()
model_predictor.delete_endpoint()

To completely shutdown SageMaker, go to File > Shut Down > Shutdown All